In [1]:
import fitz

In [2]:
doc = fitz.open("book.pdf")

In [3]:
chapter_starts_d = {"Chapter 1": 26,
                  "Chapter 2": 70,
                  "Chapter 3": 105,
                  "Chapter 4": 145,
                  "Chapter 5": 190,
                  "Chapter 6": 219,
                  "Chapter 7": 253,
                  "Chapter 8": 300,
                  "Chapter 9": 343,
                  "Chapter 10": 353,
                  "Chapter 11": 397,
                  "Chapter 12": 437,
                  "Chapter 13": 481,
                  "Chapter 14": 513,
                  "Chapter 15": 560,
                  "Chapter 16": 600,
                  "Chapter 17": 638,
                  "Chapter 18": 681,
                  "Chapter 19": 713,
                  "Chapter 20": 732,
                  "Chapter 21": 748,
                  "Out of chapter 21" : 793}
chapter_starts = list(chapter_starts_d.values())
def extract_chapters_from_pdf(pdf_path, chapter_starts):
    doc = fitz.open(pdf_path)
    chapter_dict = {}
    num_chapters = len(chapter_starts) - 1  # On suppose que la dernière entrée est la fin du dernier chapitre

    # Extraire le texte pour chaque chapitre
    for i in range(num_chapters):
        start_page = chapter_starts[i]
        end_page = chapter_starts[i + 1] - 1  # La page de fin est la page de début du prochain chapitre moins 1
        text = ""

        # Concaténer le texte de chaque page du chapitre
        for page_num in range(start_page, end_page + 1):  # +1 pour inclure la page de fin
            page = doc.load_page(page_num)
            text += page.get_text()

        chapter_dict[i + 1] = text  # Utiliser un nom générique avec numéro de chapitre

    return chapter_dict
    

In [4]:
chapter_dict = extract_chapters_from_pdf(pdf_path="book.pdf", chapter_starts=chapter_starts)

In [6]:
chapter_dict[1]

'2\u2002 \u2002 Chapter 1 / Basic Concepts and Computer Evolution \n\t 1.1\t Organization and Architecture\nIn describing computers, a distinction is often made between computer architec-\nture and computer organization. Although it is difficult to give precise definitions \nfor these terms, a consensus exists about the general areas covered by each. For \nexample, see [VRAN80], [SIEW82], and [BELL78a]; an interesting alternative view \nis presented in [REDD76].\nComputer architecture refers to those attributes of a system visible to a pro-\ngrammer or, put another way, those attributes that have a direct impact on the \nlogical execution of a program. A term that is often used interchangeably with com-\nputer architecture is instruction set architecture (ISA). The ISA defines instruction \nformats, instruction opcodes, registers, instruction and data memory; the effect of \nexecuted instructions on the registers and memory; and an algorithm for control-\nling instruction execution. Co

In [7]:
len(chapter_dict[1])

109820

In [6]:
from transformers import pipeline
def summarize_chapters(chapters):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summaries = {}
    for chapter, text in chapters.items():
        summaries[chapter] = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summaries

2024-04-28 18:25:37.832312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 18:25:37.832347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 18:25:37.857186: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 18:25:37.910734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#summaries = summarize_chapters(chapter_dict)

In [8]:
#for chapter, summary in summaries.items():
#    print(f"{chapter}: {summary}\n")

In [9]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

2024-04-28 18:25:41.859669: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 18:25:42.070338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 18:25:42.070532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
import math
def summarize_text(text):
    """
    Summarizes the provided text using a pre-initialized summarizer.
    
    Args:
    text (str): The text to summarize.

    Returns:
    str: The summarized text.
    """
    input_length = len(text.split())  # Nombre de mots dans le texte d'entrée
    max_length = max(30, math.ceil(input_length / 4))  # Ne pas descendre en dessous d'une longueur minimale, par exemple 30 mots
    min_length = max(20, math.ceil(input_length / 6))
    # Generate summary
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    
    # Return only the summary text
    return summary[0]['summary_text']

In [ ]:
text_chapter_1 = summarize_text(text=chapter_dict[1])

2024-04-28 18:25:48.758342: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51499948096 exceeds 10% of free system memory.
2024-04-28 18:25:57.952588: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51499948096 exceeds 10% of free system memory.
2024-04-28 18:27:19.997731: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51499948096 exceeds 10% of free system memory.
